# Optimization of Noise Generator

We want to find the right Hyperparameters for the Noise Generator.
For the Optimization, we will use the `Optuna` Framework

The correct Hyperparameters for the Noise will be found for:
- `GeFeU`
- `GEMU`

In [1]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import src.gefeu
import src.gemu
import src.mlp_dataclass
import src.metrics
from src.helper import load_models_dict

c:\Users\Moritz\miniconda3\envs\bach.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


___
## GeFeU

In [ ]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    print(f"Epochs: {opt_Epochs} 
          |\nLearning Rate: {opt_Learning_Rate} 
          |\nBatch Size: {opt_Batch_Size} 
          |\nN2R Ratio: {opt_N2R_Ratio} 
          |\nRegularization Term: {opt_Regularization_term} 
          |\nNoise Dim: {opt_Noise_Dim}
          |\nImpair LR: {opt_Impair_LR}
          |\nRepair LR: {opt_Repair_LR}")

    l1 = trial.suggest_int('l1', 32, 1024)
    l2 = trial.suggest_int('l2', 32, 1024)
    l3 = trial.suggest_int('l3', 32, 1024)
    l4 = trial.suggest_int('l4', 32, 1024)
    l5 = trial.suggest_int('l5', 32, 1024)
    l6 = trial.suggest_int('l6', 32, 1024)
    l7 = trial.suggest_int('l7', 32, 1024)
    l8 = trial.suggest_int('l8', 32, 1024)
    l9 = trial.suggest_int('l9', 32, 1024)
    n_layers = trial.suggest_int('n_layers', 1, 9)

    Layers = [l1, l2, l3, l4, l5, l6, l7, l8, l9]
    Layers = Layers[:n_layers]
    print("Layers: ", Layers)

    train = load_models_dict(path="data/models/mnist/all/test_ensemble")[0]
    
    unlearned = src.gefeu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=True,
        model_eval_logs=False,
    )

    valid_ds = src.mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    exact = load_models_dict(path="data/models/mnist/except_erased/test_ensemble")[0]
    
    div = src.metrics.kl_divergence_between_models(
        model1 = unlearned,
        model2 = exact,
        data_loader = valid_dl,
    )

    return div

In [ ]:
study = optuna.create_study(study_name="GenOptiGeFeU")
study.optimize(objective, n_trials=5)

study.best_params

## Standard Values

In [ ]:
# standard_model = src.gefeu._main(
#     model=None,
#     dataset_name="mnist",
#     t_Epochs=,
#     t_Batch_Size=,
#     t_Learning_Rate=,
#     t_N2R_Ratio=,
#     t_Regularization_term=,
#     t_Layers=,
#     t_Noise_Dim=,
#     t_Impair_LR=,
#     t_Repair_LR=,
#     logs=True,
#     model_eval_logs=False,
# )

___
## GEMU

In [ ]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    print(f"Epochs: {opt_Epochs} 
          |\nLearning Rate: {opt_Learning_Rate} 
          |\nBatch Size: {opt_Batch_Size} 
          |\nN2R Ratio: {opt_N2R_Ratio} 
          |\nRegularization Term: {opt_Regularization_term} 
          |\nNoise Dim: {opt_Noise_Dim}
          |\nImpair LR: {opt_Impair_LR}
          |\nRepair LR: {opt_Repair_LR}")

    l1 = trial.suggest_int('l1', 32, 1024)
    l2 = trial.suggest_int('l2', 32, 1024)
    l3 = trial.suggest_int('l3', 32, 1024)
    l4 = trial.suggest_int('l4', 32, 1024)
    l5 = trial.suggest_int('l5', 32, 1024)
    l6 = trial.suggest_int('l6', 32, 1024)
    l7 = trial.suggest_int('l7', 32, 1024)
    l8 = trial.suggest_int('l8', 32, 1024)
    l9 = trial.suggest_int('l9', 32, 1024)
    n_layers = trial.suggest_int('n_layers', 1, 9)

    Layers = [l1, l2, l3, l4, l5, l6, l7, l8, l9]
    Layers = Layers[:n_layers]
    print("Layers: ", Layers)

    train = load_models_dict(path="data/models/mnist/all/test_ensemble")[0]

    unlearned = src.gemu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=True,
        model_eval_logs=False,
    )

    valid_ds = src.mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    exact = load_models_dict(path="data/models/mnist/except_erased/test_ensemble")[0]
    
    div = src.metrics.kl_divergence_between_models(
        model1 = unlearned,
        model2 = exact,
        data_loader = valid_dl,
    )

    return div

In [ ]:
study = optuna.create_study(study_name="GenOptiGEMU")
study.optimize(objective, n_trials=5)

study.best_params

## Standard Values

In [ ]:
# standard_model = src.gefeu._main(
#     model=None,
#     dataset_name="mnist",
#     t_Epochs=,
#     t_Batch_Size=,
#     t_Learning_Rate=,
#     t_N2R_Ratio=,
#     t_Regularization_term=,
#     t_Layers=,
#     t_Noise_Dim=,
#     t_Impair_LR=,
#     t_Repair_LR=,
#     logs=True,
#     model_eval_logs=False,
# )

___